# Fine-Tuning Llama 3

In [15]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [16]:
run = wandb.init(
    project='Fine-tune Llama 3 8B simple code dataset', 
    job_type="training", 
    anonymous="allow"
)

In [17]:
base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
new_model = "llama-3-8b-simple-code"
dataset_name = "code.csv"

In [18]:
torch_dtype = torch.float16
attn_implementation = "eager"

In [19]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [24]:
print(tokenizer.chat_template)  # Check if a template is already set

{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>

'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>

' }}{% endif %}


In [25]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.chat_template = None  # Reset the chat template
model, tokenizer = setup_chat_format(model, tokenizer)  # Apply the new format

In [26]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [27]:
#Importing the dataset
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Step 1: Load your dataset
data = pd.read_csv("code.csv", delimiter=";")  # Adjust delimiter if necessary

# Step 2: Split the dataset into train and test sets
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Step 3: Save the train and test sets to CSV files
train_df.to_csv("train.csv", index=False)
test_df.to_csv("test.csv", index=False)

# Step 4: Load the datasets in Hugging Face format
train_dataset = load_dataset("csv", data_files={"train": "train.csv"})["train"]
test_dataset = load_dataset("csv", data_files={"test": "test.csv"})["test"]

instruction = """You are a programming expert specialized in reasoning and executing python algorithms. The user will provide you with a python code snippet and ask you to execute it with the given input. You should execute the code and provide the output to the user. If the code contains an error, you should identify the error and provide the user with a hint to fix it. If the code is correct, you should provide only the code output to the user."""

def format_chat_template(row):

    row_json = [{"role": "system", "content": instruction },
               {"role": "user", "content": row["python_code"]+ "\n "+ row["execution_example"]},
               {"role": "assistant", "content": row["execution_result"]}]
    
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

# Load CSV files for train and test datasets separately
train_dataset = load_dataset('csv', data_files='train.csv', header=None)
test_dataset = load_dataset('csv', data_files='test.csv', header=None)

# Manually assign column names for both train and test datasets
train_dataset = train_dataset['train'].rename_column('0', 'python_code')
train_dataset = train_dataset.rename_column('3', 'human_reasoning')
train_dataset = train_dataset.rename_column('1', 'execution_example')
train_dataset = train_dataset.rename_column('2', 'execution_result')


test_dataset = test_dataset['train'].rename_column('0', 'python_code')
test_dataset = test_dataset.rename_column('3', 'human_reasoning')
test_dataset = test_dataset.rename_column('1', 'execution_example')
test_dataset = test_dataset.rename_column('2', 'execution_result')

train_dataset = train_dataset.map(format_chat_template, num_proc= 4)
test_dataset = test_dataset.map(format_chat_template, num_proc= 4)


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map (num_proc=4):   0%|          | 0/4040 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1011 [00:00<?, ? examples/s]

In [30]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=0.1,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

In [31]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/home2/fbenslama/miniconda3/envs/deep_learning_torch/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home2/fbenslama/miniconda3/envs/deep_learning_torch/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home2/fbenslama/miniconda3/envs/deep_learning_torch/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/4040 [00:00<?, ? examples/s]

In [32]:
trainer.train()

Step,Training Loss,Validation Loss
202,0.319700,0.406570
404,0.428900,0.386251
606,0.358700,0.376752
808,0.341100,0.358209
1010,0.311900,0.352629
1212,0.359400,0.351999
1414,0.346200,0.349110
1616,0.342300,0.348421
1818,0.345000,0.347552
2020,0.361400,0.346642


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/home2/fbenslama/miniconda3/envs/deep_learning_torch/lib/python3.12/site-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/home2/fbenslama/miniconda3/envs/deep_learning_torch/lib/python3.1

TrainOutput(global_step=2020, training_loss=0.36454623385083557, metrics={'train_runtime': 2972.5799, 'train_samples_per_second': 1.359, 'train_steps_per_second': 0.68, 'total_flos': 2.478899687559168e+16, 'train_loss': 0.36454623385083557, 'epoch': 1.0})

In [33]:
wandb.finish()
model.config.use_cache = True

eval/loss,█▆▅▂▂▂▁▁▁▁
eval/runtime,█▆▅▁▃█▃▂▅▅
eval/samples_per_second,▁▃▄█▆▁▆▇▄▄
eval/steps_per_second,▁▃▄█▆▁▆▇▄▄
train/epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇█
train/global_step,▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
train/grad_norm,▆▂▂▂▂▂▁▂▄█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,███████▇▇▇▇▇▇▇▇▆▆▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁
train/loss,▆▇▇▅█▄▃▆▅▅▅▆▄▄▄▁▅▅▄▅▃▅▅▅▄▃▅▂▂▅▅▅▄▅▄▄▄▄▁▂
eval/loss,0.34664
eval/runtime,142.3255
